In [1]:
from __future__ import division
import ROOT
from ROOT import gSystem, gInterpreter
import numpy as np
import matplotlib as mpl
mpl.use('Agg')
from matplotlib import pyplot as plt
import pandas as pd
from tqdm import tqdm
import matplotlib.cm as cm
from matplotlib.patches import Circle, Wedge, RegularPolygon
from matplotlib.collections import PatchCollection
from multiprocessing import Pool, cpu_count
import math
import gc
import os
import sys
from collections import defaultdict
import warnings
warnings.filterwarnings("ignore")

mpl.interactive(False)

Welcome to JupyROOT 6.20/04


In [3]:
gInterpreter.AddIncludePath("/home/felipe/MG5_aMC_v2_7_2_py3/Delphes")
gInterpreter.AddIncludePath("/home/felipe/MG5_aMC_v2_7_2_py3/Delphes/external/") 
gInterpreter.Declare('#include "/home/felipe/MG5_aMC_v2_7_2_py3/Delphes/classes/DelphesClasses.h"') 
gInterpreter.Declare('#include "/home/felipe/MG5_aMC_v2_7_2_py3/Delphes/external/ExRootAnalysis/ExRootAnalysisLinkDef.h"') 
gSystem.Load("/home/felipe/MG5_aMC_v2_7_2_py3/Delphes/libDelphes.so") 

1

In [4]:
from ROOT import Tower, Track

#Signal and Backgrounds
# W_plus_jets = ROOT.TChain("Delphes")
# chain_list = [W_plus_jets]

Z_plus_jets = ROOT.TChain("Delphes")
chain_list = [Z_plus_jets]

In [5]:
#Add corresponding data to the Signal and backgrounds chains
Events = {'Z_plus_jets':'/media/felipe/EXTERNAL_USB/1-BGL-ML_roots/1_ROOTS/Z_plus_jets/Z_plus_jets_3.root'}
for i,j in enumerate(Events.items()):
    chain_list[i].Add(j[1])
    print('number of events {}:'.format(j[0]),chain_list[i].GetEntries())


number of events Z_plus_jets: 99618


In [6]:
class TestImage:
    
    def __init__(self, img_size=128, path=''):
        self.img_size = img_size
        self.path = path
        self.eta_min = -5.0
        self.eta_max = 5.0
        self.phi_min = -np.pi
        self.phi_max = np.pi
        self.shape_scale = 0.3
        
        self.extent = self.eta_min, self.eta_max, -np.pi, np.pi
        self.bins = img_size
        self.eta_edges = np.linspace(self.eta_min, self.eta_max, self.bins + 1)
        self.phi_edges = np.linspace(-np.pi, np.pi, self.bins + 1)
        
    def abstract_images(self, coords_track, coords_photon, coords_neutral, idx, fc='w', lw=6.0):
        fig, ax = plt.subplots(subplot_kw=dict(xlim=(self.eta_min,self.eta_max),
                                               ylim=(self.phi_min,self.phi_max)),
                               figsize=(self.img_size/10, self.img_size/10),dpi=10, facecolor=fc)
        ax.axis('off')
        ax.set_facecolor(fc)
        
        
        for (i1,j1,k1),(i2,j2,k2),(i3,j3,k3) in zip(coords_track,coords_photon,coords_neutral):
            circle = Circle((i1,j1), self.shape_scale*math.log(k1),
                               fill=False, lw=lw, color='r')
            square = RegularPolygon((i2,j2), 4, self.shape_scale*math.log(k2),
                           fill=False, lw=lw, color='g')
            hexagon = RegularPolygon((i3,j3), 6, self.shape_scale*math.log(k3),
                           fill=False, lw=lw, color='b')
            ax.add_artist(circle)
            ax.add_artist(square)
            ax.add_artist(hexagon)
            
        plt.tight_layout()
        plt.savefig(self.path + '/{}.png'.format(idx), dpi=10, facecolor=ax.get_facecolor())
        plt.clf()
        plt.close('all')


In [7]:
def get_coords(event):
    tracks = list(filter(lambda Tracks: np.abs(Tracks.Eta) <= 5., event.EFlowTrack))
    Photons = list(filter(lambda Tracks: np.abs(Tracks.Eta) <= 5., event.EFlowPhoton))
    Neutral = list(filter(lambda Tracks: np.abs(Tracks.Eta) <= 5., event.EFlowNeutralHadron))
    
    coords_tracks = [(el.Eta, el.Phi-2.*np.pi if el.Phi > np.pi else el.Phi, el.PT) for el in tracks]
    coords_Photons = [(el.Eta, el.Phi-2.*np.pi if el.Phi > np.pi else el.Phi, el.ET) for el in Photons]
    coords_Neutral = [(el.Eta, el.Phi-2.*np.pi if el.Phi > np.pi else el.Phi, el.ET) for el in Neutral]
    
    return coords_tracks, coords_Photons, coords_Neutral


In [8]:
def get_charged_leptons(event):
    
    electrons = list(filter(lambda Electrons: np.abs(Electrons.PID) in [11], event.Particle))
    electrons = list(filter(lambda Electrons: np.abs(Electrons.Eta) <= 2.5 and Electrons.PT > 25., electrons))
    electrons = list(filter(lambda Electrons: Electrons.PT > 25., electrons))
    
    muons = list(filter(lambda Muons: np.abs(Muons.PID) in [13], event.Particle))
    muons = list(filter(lambda Muons: np.abs(Muons.Eta) <= 2.5 and Muons.PT > 25., muons))
    muons = list(filter(lambda Muons: Muons.PT > 25., muons))

        
    Nu_event = list(filter(lambda NU: np.abs(NU.PID) in [12,14,16], event.Particle))
    Nu_event = list(filter(lambda NU: NU.PT > 20., Nu_event))
    MET = Nu_event
    
    leptons = muons + electrons
        
    return leptons, MET


In [9]:
def get_jets(event):
    jets = list(filter(lambda Jets: np.abs(Jets.Eta) <= 2.5 and Jets.PT > 25., event.Jet))
    jets = sorted(jets, key=lambda Jets: Jets.PT, reverse=True)
    bjets, ljets = [], []
    nbjets = 0
    for n in range(len(jets)):
        if jets[n].BTag == 1:
            bjets.append(jets[n])
            nbjets += 1
        else:
            ljets.append(jets[n])         
            
    b_jets = bjets
    l_jets = ljets
    
    return l_jets, b_jets


In [10]:
# call the class here and define the path:
for chain, proc in zip(chain_list,Events.items()):
    
    df = pd.DataFrame(columns=['event id', 'weight', 'pT_lepton', 'eta_lepton', 'phi_lepton',
                                           'Energy_lepton', 'met', 'pT_jet1', 'eta_jet1', 'phi_jet1',
                                           'Energy_jet1', 'pT_jet2', 'eta_jet2', 'phi_jet2', 'Energy_jet2',
                                           'pT_jet3', 'eta_jet3', 'phi_jet3', 'Energy_jet3', 'pT_jet4',
                                           'eta_jet4', 'phi_jet4', 'Energy_jet4', 'MT_W', 'MT_W_jets','MD',
                                           'MDanti', 'MD1', 'MD2', 'MJ1J2', 'MJ1J3', 'MJ1J4', 'MJ2J3', 'MJ2J4',
                                           'MJ3J4', 'DR_j1lepton', 'DR_j2lepton', 'DR_j3lepton', 'DR_j4lepton',
                                           'DR_j1j2', 'DR_j1j3', 'DR_j1j4', 'DR_j2j3', 'DR_j2j4', 'DR_j3j4',
                                           'DPhi_j1lepton', 'DPhi_j2lepton', 'DPhi_j3lepton', 'DPhi_j4lepton',
                                           'DPhi_j1j2', 'DPhi_j1j3', 'DPhi_j1j4', 'DPhi_j2j3', 'DPhi_j2j4',
                                           'DPhi_j3j4', 'cos_lepton_MET', 'cos_lepton_jet1', 'cos_lepton_jet2',
                                           'cos_lepton_jet3', 'cos_lepton_jet4', 'cos_jet1_jet2', 'cos_jet1_jet3',
                                           'cos_jet1_jet4', 'cos_jet2_jet3', 'cos_jet2_jet4', 'cos_jet3_jet4'])
    
    for count, event in enumerate(tqdm(chain, total=chain.GetEntries())):
    
        #if count == 1000:
        #    break
       
        #Get particles
        leptons, MET = get_charged_leptons(event)
        ljets, bjets = get_jets(event)
        
        #Four vectors
        #At least one lepton, 4 light jets, one missing vector
        
        lepton_vec = ROOT.TLorentzVector()
        MET_vec = ROOT.TLorentzVector()
        
        l1jet_vec = ROOT.TLorentzVector()
        l2jet_vec = ROOT.TLorentzVector()
        l3jet_vec = ROOT.TLorentzVector()
        l4jet_vec = ROOT.TLorentzVector()

    
        if (leptons and MET and len(ljets) > 3):
            lepton_vec.SetPtEtaPhiE(leptons[0].PT,leptons[0].Eta,leptons[0].Phi,leptons[0].P4().E())
            MET_vec.SetPtEtaPhiE(MET[0].P4().Pt(),MET[0].Eta,MET[0].Phi,MET[0].P4().E())
            
            l1jet_vec.SetPtEtaPhiE(ljets[0].P4().Pt(),ljets[0].Eta,ljets[0].Phi,ljets[0].P4().E())
            l2jet_vec.SetPtEtaPhiE(ljets[1].P4().Pt(),ljets[1].Eta,ljets[1].Phi,ljets[1].P4().E())
            l3jet_vec.SetPtEtaPhiE(ljets[2].P4().Pt(),ljets[2].Eta,ljets[2].Phi,ljets[2].P4().E())
            l4jet_vec.SetPtEtaPhiE(ljets[3].P4().Pt(),ljets[3].Eta,ljets[3].Phi,ljets[3].P4().E())     
            
            #Lepton observables
            pT_lepton = lepton_vec.Pt()
            eta_lepton = lepton_vec.Eta()
            phi_lepton = lepton_vec.Phi()
            Energy_lepton = lepton_vec.E()    
            
            #MET
            met = MET[0].P4().Mt()
            
            #Jet observables
            pT_jet1 = l1jet_vec.Pt()
            eta_jet1 = l1jet_vec.Eta()
            phi_jet1 = l1jet_vec.Phi()
            Energy_jet1 = l1jet_vec.E()
            
            pT_jet2 = l2jet_vec.Pt()
            eta_jet2 = l2jet_vec.Eta()
            phi_jet2 = l2jet_vec.Phi()
            Energy_jet2 = l2jet_vec.E()

            
            pT_jet3 = l3jet_vec.Pt()
            eta_jet3 = l3jet_vec.Eta()
            phi_jet3 = l3jet_vec.Phi()
            Energy_jet3 = l3jet_vec.E()

            
            pT_jet4 = l4jet_vec.Pt()
            eta_jet4 = l4jet_vec.Eta()
            phi_jet4 = l4jet_vec.Phi()
            Energy_jet4 = l4jet_vec.E()
            
            #Reconstructed observables
            #Invariant masses of the W's
            
            #From the leptons
            W_lep = MET_vec + lepton_vec
            pt_miss = MET_vec.Pt()
            theta = lepton_vec.DeltaPhi(MET_vec)
            MT_W = np.sqrt(2*pT_lepton*pt_miss*(1- np.cos(theta)))
            
            #From the two jets
            W_jets = (l3jet_vec + l4jet_vec)
            MT_W_jets = W_jets.M()
            
            #VLQs
            Dquark = W_lep + l1jet_vec
            AntiDquark = W_jets + l2jet_vec
            MD = Dquark.M()
            MDanti = AntiDquark.M()
            
            #Invariant masses of different combinations
            MD1 = (W_lep + l2jet_vec).M()
            MD2 = (W_jets + l1jet_vec).M()
            
            #Invariant masses for different combinations of two-jets
            MJ1J2 = (l1jet_vec + l2jet_vec).M()
            MJ1J3 = (l1jet_vec + l3jet_vec).M()
            MJ1J4 = (l1jet_vec + l4jet_vec).M()
            MJ2J3 = (l2jet_vec + l3jet_vec).M()
            MJ2J4 = (l2jet_vec + l4jet_vec).M()
            MJ3J4 = (l3jet_vec + l4jet_vec).M()
            
            #Delta R
            DR_j1lepton = lepton_vec.DeltaR(l1jet_vec)
            DR_j2lepton = lepton_vec.DeltaR(l2jet_vec)
            DR_j3lepton = lepton_vec.DeltaR(l3jet_vec)
            DR_j4lepton = lepton_vec.DeltaR(l3jet_vec)
            
            DR_j1j2 = l1jet_vec.DeltaR(l2jet_vec)
            DR_j1j3 = l1jet_vec.DeltaR(l3jet_vec)
            DR_j1j4 = l1jet_vec.DeltaR(l4jet_vec)
            
            DR_j2j3 = l2jet_vec.DeltaR(l3jet_vec)
            DR_j2j4 = l2jet_vec.DeltaR(l4jet_vec)            
            
            DR_j3j4 = l3jet_vec.DeltaR(l4jet_vec)
            
            #Delta phi
            DPhi_j1lepton = lepton_vec.DeltaPhi(l1jet_vec)
            DPhi_j2lepton = lepton_vec.DeltaPhi(l2jet_vec)
            DPhi_j3lepton = lepton_vec.DeltaPhi(l3jet_vec)
            DPhi_j4lepton = lepton_vec.DeltaPhi(l3jet_vec)
            
            DPhi_j1j2 = l1jet_vec.DeltaPhi(l2jet_vec)
            DPhi_j1j3 = l1jet_vec.DeltaPhi(l3jet_vec)
            DPhi_j1j4 = l1jet_vec.DeltaPhi(l4jet_vec)
            
            DPhi_j2j3 = l2jet_vec.DeltaPhi(l3jet_vec)
            DPhi_j2j4 = l2jet_vec.DeltaPhi(l4jet_vec)            
            
            DPhi_j3j4 = l3jet_vec.DeltaPhi(l4jet_vec)            

            #Cosine of angles
            T3_lepton = lepton_vec.Vect()
            T3_MET = MET_vec.Vect()
            T3_j1 = l1jet_vec.Vect()
            T3_j2 = l2jet_vec.Vect()
            T3_j3 = l3jet_vec.Vect()
            T3_j4 = l4jet_vec.Vect()
                
            
            cos_lepton_MET = np.cos(T3_lepton.Angle(T3_MET))
            cos_lepton_jet1 = np.cos(T3_lepton.Angle(T3_j1))
            cos_lepton_jet2 = np.cos(T3_lepton.Angle(T3_j2))
            cos_lepton_jet3 = np.cos(T3_lepton.Angle(T3_j3))
            cos_lepton_jet4 = np.cos(T3_lepton.Angle(T3_j4))
            
            cos_jet1_jet2 = np.cos(T3_j1.Angle(T3_j2))
            cos_jet1_jet3 = np.cos(T3_j1.Angle(T3_j3))
            cos_jet1_jet4 = np.cos(T3_j1.Angle(T3_j4))

            cos_jet2_jet3 = np.cos(T3_j2.Angle(T3_j3))
            cos_jet2_jet4 = np.cos(T3_j2.Angle(T3_j4))

            cos_jet3_jet4 = np.cos(T3_j3.Angle(T3_j4))
            
            event_weight = [eevent.Weight for eevent in event.Event]
            xs_weight = event_weight[0] if len(event_weight) == 1 else 0

            entry = pd.DataFrame([[count,xs_weight,pT_lepton,eta_lepton, phi_lepton,
                                           Energy_lepton, met, pT_jet1, eta_jet1, phi_jet1,
                                           Energy_jet1, pT_jet2, eta_jet2, phi_jet2, Energy_jet2,
                                           pT_jet3, eta_jet3, phi_jet3, Energy_jet3, pT_jet4,
                                           eta_jet4, phi_jet4, Energy_jet4, MT_W, MT_W_jets, MD,
                                           MDanti, MD1, MD2, MJ1J2, MJ1J3, MJ1J4, MJ2J3, MJ2J4,
                                           MJ3J4, DR_j1lepton, DR_j2lepton, DR_j3lepton, DR_j4lepton,
                                           DR_j1j2, DR_j1j3, DR_j1j4, DR_j2j3, DR_j2j4, DR_j3j4,
                                           DPhi_j1lepton, DPhi_j2lepton, DPhi_j3lepton, DPhi_j4lepton,
                                           DPhi_j1j2, DPhi_j1j3, DPhi_j1j4, DPhi_j2j3, DPhi_j2j4,
                                           DPhi_j3j4, cos_lepton_MET, cos_lepton_jet1, cos_lepton_jet2,
                                           cos_lepton_jet3, cos_lepton_jet4, cos_jet1_jet2, cos_jet1_jet3,
                                           cos_jet1_jet4, cos_jet2_jet3, cos_jet2_jet4, cos_jet3_jet4]],                                  
                                  columns=['event id', 'weight', 'pT_lepton', 'eta_lepton', 'phi_lepton',
                                           'Energy_lepton', 'met', 'pT_jet1', 'eta_jet1', 'phi_jet1',
                                           'Energy_jet1', 'pT_jet2', 'eta_jet2', 'phi_jet2', 'Energy_jet2',
                                           'pT_jet3', 'eta_jet3', 'phi_jet3', 'Energy_jet3', 'pT_jet4',
                                           'eta_jet4', 'phi_jet4', 'Energy_jet4', 'MT_W', 'MT_W_jets','MD',
                                           'MDanti', 'MD1', 'MD2', 'MJ1J2', 'MJ1J3', 'MJ1J4', 'MJ2J3', 'MJ2J4',
                                           'MJ3J4', 'DR_j1lepton', 'DR_j2lepton', 'DR_j3lepton', 'DR_j4lepton',
                                           'DR_j1j2', 'DR_j1j3', 'DR_j1j4', 'DR_j2j3', 'DR_j2j4', 'DR_j3j4',
                                           'DPhi_j1lepton', 'DPhi_j2lepton', 'DPhi_j3lepton', 'DPhi_j4lepton',
                                           'DPhi_j1j2', 'DPhi_j1j3', 'DPhi_j1j4', 'DPhi_j2j3', 'DPhi_j2j4',
                                           'DPhi_j3j4', 'cos_lepton_MET', 'cos_lepton_jet1', 'cos_lepton_jet2',
                                           'cos_lepton_jet3', 'cos_lepton_jet4', 'cos_jet1_jet2', 'cos_jet1_jet3',
                                           'cos_jet1_jet4', 'cos_jet2_jet3', 'cos_jet2_jet4', 'cos_jet3_jet4'])
    
            df = df.append(entry)    
            df.to_csv('/home/felipe/JoaoPino/vlq_joao_StrongProduction/TabularData/{}_3.csv'.format(proc[0]), sep=',', index=False)
            
                
            #images    
            c1, c2, c3 = get_coords(event)
            
            timage = TestImage(path='/home/felipe/JoaoPino/vlq_joao_StrongProduction/Images_Abstract/{}'.format(proc[0]),img_size=224)
            timage.abstract_images(c1, c2, c3, count)


100%|██████████| 99618/99618 [22:47<00:00, 72.83it/s]  
